<a href="https://colab.research.google.com/github/Kalit31/Data-Mining-Assignment/blob/main/LR%26SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import necessary packages
import pandas as pd
import numpy as np


from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import  LogisticRegression
from sklearn import svm

from scipy.sparse import coo_matrix, hstack

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_pre=pd.read_csv('/content/gdrive/My Drive/DM project/data_preprocessed.csv')

In [ ]:
traindata=df_pre[df_pre['train-test-val']==0]
print(traindata.shape)
valdata=df_pre[df_pre['train-test-val']==1]
print(valdata.shape)
testdata=df_pre[df_pre['train-test-val']==2]
print(testdata.shape)

(10240, 289)
(1284, 289)
(1267, 289)


In [ ]:
# traindata.reset_index(drop=True)
# valdata.reset_index(drop=True)
# testdata.reset_index(drop=True)

In [ ]:
labelsList=['barely-true','false','half-true','mostly-true','pants-fire','true']
is_fakeList=['false','true']

# Functions

In [ ]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

# **Six-Way Classification**

## S condition

In [ ]:
#unigram features
cvec = CountVectorizer()
X_train = cvec.fit_transform(traindata['Statement'])
X_val = cvec.transform(valdata['Statement'])
X_test = cvec.transform(testdata['Statement'])

In [ ]:
#tf-idf weighted unigram features
tfidf = TfidfVectorizer()
Xt_train = tfidf.fit_transform(traindata['Statement'])
Xt_val = tfidf.transform(valdata['Statement'])
Xt_test = tfidf.transform(testdata['Statement'])

In [ ]:
# Glove word embeddings
word2vec_output_file = 'glove.6B.50d' +'.word2vec'
glove2word2vec('/content/gdrive/My Drive/DM project/glove.6B.50d.txt', word2vec_output_file)
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
vectorizer = Word2VecVectorizer(model)

Loading in word vectors...
Finished loading in word vectors


In [ ]:
Xg_train = vectorizer.fit_transform(traindata['Statement'])
print(Xg_train.shape)
Xg_val = vectorizer.transform(valdata['Statement'])
Xg_test = vectorizer.transform(testdata['Statement'])

Numer of samples with no words found: 0 / 7212
(7212, 300)
Numer of samples with no words found: 0 / 929
Numer of samples with no words found: 0 / 924


### SVM

In [ ]:
SVM = svm.LinearSVC()

In [ ]:
SVM.fit(X_train, traindata['Label']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['Label'], SVM.predict(X_val),target_names=labelsList,digits=4))
print(np.mean(SVM.predict(X_val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2476    0.2194    0.2327       237
       false     0.2963    0.3042    0.3002       263
   half-true     0.2435    0.2661    0.2543       248
 mostly-true     0.2355    0.2271    0.2312       251
  pants-fire     0.2062    0.1724    0.1878       116
        true     0.2113    0.2426    0.2259       169

    accuracy                         0.2461      1284
   macro avg     0.2401    0.2386    0.2387      1284
weighted avg     0.2459    0.2461    0.2455      1284

0.24610591900311526


In [ ]:
print(classification_report(testdata['Label'], SVM.predict(X_test),target_names=labelsList,digits=4))
np.mean(SVM.predict(X_test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2207    0.2119    0.2162       151
       false     0.2747    0.2732    0.2740       183
   half-true     0.2295    0.2143    0.2216       196
 mostly-true     0.2564    0.2717    0.2639       184
  pants-fire     0.1071    0.1176    0.1121        51
        true     0.1902    0.1950    0.1925       159

    accuracy                         0.2284       924
   macro avg     0.2131    0.2140    0.2134       924
weighted avg     0.2289    0.2284    0.2285       924



0.22835497835497837

In [ ]:
SVM.fit(Xt_train, traindata['Label'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['Label'], SVM.predict(Xt_val),target_names=labelsList,digits=4))
np.mean(SVM.predict(Xt_val)== valdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2119    0.1893    0.2000       169
       false     0.2376    0.2553    0.2462       188
   half-true     0.2361    0.2772    0.2550       184
 mostly-true     0.2556    0.2460    0.2507       187
  pants-fire     0.3030    0.1408    0.1923        71
        true     0.1905    0.2154    0.2022       130

    accuracy                         0.2314       929
   macro avg     0.2391    0.2207    0.2244       929
weighted avg     0.2347    0.2314    0.2302       929



0.23143164693218515

In [ ]:
print(classification_report(testdata['Label'], SVM.predict(Xt_test),target_names=labelsList,digits=4))
np.mean(SVM.predict(Xt_test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2061    0.1788    0.1915       151
       false     0.2717    0.2732    0.2725       183
   half-true     0.2362    0.2398    0.2380       196
 mostly-true     0.2244    0.2500    0.2365       184
  pants-fire     0.1628    0.1373    0.1489        51
        true     0.2160    0.2201    0.2181       159

    accuracy                         0.2294       924
   macro avg     0.2195    0.2165    0.2176       924
weighted avg     0.2284    0.2294    0.2286       924



0.22943722943722944

In [ ]:
SVM.fit(Xg_train, traindata['Label'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(testdata['Label'], SVM.predict(Xg_test),target_names=labelsList,digits=4))
np.mean(SVM.predict(Xg_test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2532    0.1325    0.1739       151
       false     0.2610    0.3552    0.3009       183
   half-true     0.2458    0.3010    0.2706       196
 mostly-true     0.2629    0.3043    0.2821       184
  pants-fire     0.1333    0.0392    0.0606        51
        true     0.2578    0.2075    0.2300       159

    accuracy                         0.2543       924
   macro avg     0.2357    0.2233    0.2197       924
weighted avg     0.2493    0.2543    0.2445       924



0.25432900432900435

In [ ]:
print(classification_report(valdata['Label'], SVM.predict(Xg_val),target_names=labelsList,digits=4))
np.mean(SVM.predict(Xg_val)== valdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1905    0.1183    0.1460       169
       false     0.2759    0.3404    0.3048       188
   half-true     0.2161    0.2772    0.2429       184
 mostly-true     0.2044    0.2460    0.2233       187
  pants-fire     0.1818    0.0282    0.0488        71
        true     0.2083    0.1923    0.2000       130

    accuracy                         0.2239       929
   macro avg     0.2128    0.2004    0.1943       929
weighted avg     0.2175    0.2239    0.2130       929



0.2238966630785791

### LR

In [ ]:
lr = LogisticRegression()


In [ ]:
lr.fit(X_train, traindata['Label']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['Label'], lr.predict(X_val),target_names=labelsList,digits=4))
print(np.mean(lr.predict(X_val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2400    0.2130    0.2257       169
       false     0.2217    0.2606    0.2396       188
   half-true     0.2383    0.2500    0.2440       184
 mostly-true     0.2626    0.2513    0.2568       187
  pants-fire     0.3333    0.1549    0.2115        71
        true     0.1895    0.2231    0.2049       130

    accuracy                         0.2347       929
   macro avg     0.2476    0.2255    0.2304       929
weighted avg     0.2406    0.2347    0.2344       929

0.23466092572658773


In [ ]:
print(classification_report(testdata['Label'], lr.predict(X_test),target_names=labelsList,digits=4))
np.mean(lr.predict(X_test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2177    0.1788    0.1964       151
       false     0.2660    0.2951    0.2798       183
   half-true     0.2289    0.2347    0.2317       196
 mostly-true     0.2438    0.2663    0.2545       184
  pants-fire     0.0588    0.0392    0.0471        51
        true     0.2174    0.2201    0.2187       159

    accuracy                         0.2305       924
   macro avg     0.2054    0.2057    0.2047       924
weighted avg     0.2260    0.2305    0.2276       924



0.2305194805194805

In [ ]:
lr.fit(Xt_train, traindata['Label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['Label'], lr.predict(Xt_val),target_names=labelsList,digits=4))
np.mean(lr.predict(Xt_val)== valdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2255    0.1361    0.1697       169
       false     0.2428    0.3138    0.2738       188
   half-true     0.2325    0.2880    0.2573       184
 mostly-true     0.2558    0.2941    0.2736       187
  pants-fire     1.0000    0.0141    0.0278        71
        true     0.1643    0.1769    0.1704       130

    accuracy                         0.2304       929
   macro avg     0.3535    0.2038    0.1954       929
weighted avg     0.2871    0.2304    0.2183       929



0.2303552206673843

In [ ]:
print(classification_report(testdata['Label'], lr.predict(Xt_test),target_names=labelsList,digits=4))
np.mean(lr.predict(Xt_test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2796    0.1722    0.2131       151
       false     0.2743    0.3388    0.3032       183
   half-true     0.2510    0.3061    0.2759       196
 mostly-true     0.2437    0.3152    0.2749       184
  pants-fire     0.0000    0.0000    0.0000        51
        true     0.2381    0.1887    0.2105       159

    accuracy                         0.2554       924
   macro avg     0.2145    0.2202    0.2129       924
weighted avg     0.2428    0.2554    0.2444       924



0.2554112554112554

In [ ]:
lr.fit(Xg_train, traindata['Label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(testdata['Label'], lr.predict(Xg_test),target_names=labelsList,digits=4))
np.mean(lr.predict(Xg_test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.2174    0.1325    0.1646       151
       false     0.2672    0.3388    0.2988       183
   half-true     0.2500    0.3163    0.2793       196
 mostly-true     0.2388    0.2609    0.2494       184
  pants-fire     0.1667    0.0588    0.0870        51
        true     0.2632    0.2201    0.2397       159

    accuracy                         0.2489       924
   macro avg     0.2339    0.2212    0.2198       924
weighted avg     0.2435    0.2489    0.2410       924



0.24891774891774893

In [ ]:
print(classification_report(valdata['Label'], lr.predict(Xg_val),target_names=labelsList,digits=4))
np.mean(lr.predict(Xg_val)== valdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1560    0.1006    0.1223       169
       false     0.2798    0.3245    0.3005       188
   half-true     0.2273    0.2989    0.2582       184
 mostly-true     0.2300    0.2620    0.2450       187
  pants-fire     0.1111    0.0282    0.0449        71
        true     0.2016    0.2000    0.2008       130

    accuracy                         0.2260       929
   macro avg     0.2010    0.2024    0.1953       929
weighted avg     0.2130    0.2260    0.2150       929



0.22604951560818085

## SJ


In [ ]:
svec = CountVectorizer()
sTrain = svec.fit_transform(traindata['Statement'])
sVal = svec.transform(valdata['Statement'])
sTest = svec.transform(testdata['Statement'])

jvec = CountVectorizer()
jTrain = jvec.fit_transform(traindata['Justification'])
jVal = jvec.transform(valdata['Justification'])
jTest = jvec.transform(testdata['Justification'])

train=hstack([sTrain,jTrain])
val=hstack([sVal,jVal])
test=hstack([sTest,jTest])

### SVM

In [ ]:
SVM=svm.LinearSVC()

In [ ]:
SVM.fit(train, traindata['Label']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['Label'], SVM.predict(val),target_names=labelsList,digits=4))
print(np.mean(SVM.predict(val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2513    0.2110    0.2294       237
       false     0.1946    0.1901    0.1923       263
   half-true     0.2470    0.2500    0.2485       248
 mostly-true     0.2753    0.2709    0.2731       251
  pants-fire     0.1702    0.1379    0.1524       116
        true     0.1695    0.2367    0.1975       169

    accuracy                         0.2227      1284
   macro avg     0.2180    0.2161    0.2155      1284
weighted avg     0.2254    0.2227    0.2229      1284

0.22274143302180685


In [ ]:
print(classification_report(testdata['Label'], SVM.predict(test),target_names=labelsList,digits=4))
np.mean(SVM.predict(test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1778    0.1887    0.1831       212
       false     0.2151    0.2169    0.2160       249
   half-true     0.2285    0.2302    0.2293       265
 mostly-true     0.2105    0.1992    0.2047       241
  pants-fire     0.2250    0.1957    0.2093        92
        true     0.1944    0.2019    0.1981       208

    accuracy                         0.2076      1267
   macro avg     0.2086    0.2054    0.2067      1267
weighted avg     0.2081    0.2076    0.2077      1267



0.20757695343330704

### LR


In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(train, traindata['Label']) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['Label'], lr.predict(val),target_names=labelsList,digits=4))
print(np.mean(lr.predict(val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2066    0.1479    0.1724       169
       false     0.2512    0.2766    0.2633       188
   half-true     0.2406    0.2446    0.2426       184
 mostly-true     0.2919    0.3262    0.3081       187
  pants-fire     0.1556    0.0986    0.1207        71
        true     0.1875    0.2308    0.2069       130

    accuracy                         0.2368       929
   macro avg     0.2222    0.2208    0.2190       929
weighted avg     0.2330    0.2368    0.2329       929

0.23681377825618946


In [ ]:
print(classification_report(testdata['Label'], lr.predict(test),target_names=labelsList,digits=4))
np.mean(lr.predict(test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1947    0.1457    0.1667       151
       false     0.2476    0.2842    0.2646       183
   half-true     0.2249    0.2398    0.2321       196
 mostly-true     0.2670    0.2989    0.2821       184
  pants-fire     0.1379    0.0784    0.1000        51
        true     0.2484    0.2453    0.2468       159

    accuracy                         0.2370       924
   macro avg     0.2201    0.2154    0.2154       924
weighted avg     0.2321    0.2370    0.2330       924



0.237012987012987

### Testing ensemble

In [ ]:
#testing ensemble stacking technique
lr.fit(sTrain, traindata['Label']) 
Str=pd.DataFrame(lr.decision_function(sTrain),columns=['S_' + sub for sub in labelsList])
S=pd.DataFrame(lr.decision_function(sVal),columns=['S_' + sub for sub in labelsList])


lr.fit(jTrain, traindata['Label']) 
Jtr=pd.DataFrame(lr.decision_function(jTrain),columns=['J_' + sub for sub in labelsList])
J=pd.DataFrame(lr.decision_function(jVal),columns=['J_' + sub for sub in labelsList])

In [ ]:
df1=pd.concat([Str,Jtr],axis=1)
df2=pd.concat([S,J],axis=1)

In [ ]:
lr_meta=LogisticRegression()
lr_meta.fit(df1,traindata['Label'])
pred=lr_meta.predict(df2)

print(classification_report(valdata['Label'], pred,target_names=labelsList,digits=4))
print(np.mean(pred== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2245    0.1857    0.2032       237
       false     0.2482    0.2662    0.2569       263
   half-true     0.2465    0.2823    0.2632       248
 mostly-true     0.3056    0.3068    0.3062       251
  pants-fire     0.2692    0.1810    0.2165       116
        true     0.1927    0.2189    0.2050       169

    accuracy                         0.2484      1284
   macro avg     0.2478    0.2401    0.2418      1284
weighted avg     0.2493    0.2484    0.2473      1284

0.2484423676012461


## S+M

In [ ]:
meta_features =['Context', 'Subject', 'Job Title']
vec_St=CountVectorizer()
vec_C=CountVectorizer()
vec_S=CountVectorizer()
vec_Jt=CountVectorizer()

Train_St = vec_St.fit_transform(traindata['Statement'])
Val_St = vec_St.transform(valdata['Statement'])
Test_St = vec_St.transform(testdata['Statement'])

Train_C = vec_C.fit_transform(traindata['Context'])
Val_C = vec_C.transform(valdata['Context'])
Test_C = vec_C.transform(testdata['Context'])

Train_S = vec_S.fit_transform(traindata['Subject'])
Val_S = vec_S.transform(valdata['Subject'])
Test_S = vec_S.transform(testdata['Subject'])

Train_Jt = vec_Jt.fit_transform(traindata['Job Title'])
Val_Jt = vec_Jt.transform(valdata['Job Title'])
Test_Jt = vec_Jt.transform(testdata['Job Title'])

train=hstack([Train_St,Train_C,Train_S,Train_Jt])
val=hstack([Val_St,Val_C,Val_S,Val_Jt])
test=hstack([Test_St,Test_C,Test_S,Test_Jt])

train= hstack([train,traindata[traindata.columns[17:]]])
val= hstack([val,valdata[valdata.columns[17:]]])
test= hstack([test,testdata[testdata.columns[17:]]])


mTrain=hstack([Train_C,Train_S,Train_Jt])
mVal=hstack([Val_C,Val_S,Val_Jt])
mTest=hstack([Test_C,Test_S,Test_Jt])

### SVM

In [ ]:
SVM = svm.LinearSVC()

In [ ]:
SVM.fit(train, traindata['Label']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['Label'], SVM.predict(val),target_names=labelsList,digits=4))
print(np.mean(SVM.predict(val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.1789    0.1302    0.1507       169
       false     0.2269    0.2606    0.2426       188
   half-true     0.2896    0.2880    0.2888       184
 mostly-true     0.2819    0.2834    0.2827       187
  pants-fire     0.1698    0.1268    0.1452        71
        true     0.1867    0.2385    0.2095       130

    accuracy                         0.2336       929
   macro avg     0.2223    0.2213    0.2199       929
weighted avg     0.2317    0.2336    0.2310       929

0.23358449946178686


In [ ]:
print(classification_report(testdata['Label'], SVM.predict(test),target_names=labelsList,digits=4))
np.mean(SVM.predict(test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1385    0.1192    0.1281       151
       false     0.1977    0.1913    0.1944       183
   half-true     0.2195    0.2296    0.2244       196
 mostly-true     0.2559    0.2935    0.2734       184
  pants-fire     0.0750    0.0588    0.0659        51
        true     0.2236    0.2264    0.2250       159

    accuracy                         0.2067       924
   macro avg     0.1850    0.1865    0.1852       924
weighted avg     0.2019    0.2067    0.2039       924



0.2067099567099567

### Testing ensemble

In [ ]:
lr.fit(sTrain, traindata['Label']) 
Str=pd.DataFrame(lr.decision_function(sTrain),columns=['S_' + sub for sub in labelsList])
S=pd.DataFrame(lr.decision_function(sVal),columns=['S_' + sub for sub in labelsList])

#training on metadata countVectors
lr.fit(mTrain, traindata['Label']) 
Mvtr=pd.DataFrame(lr.decision_function(mTrain),columns=['Mv_' + sub for sub in labelsList])
Mv=pd.DataFrame(lr.decision_function(mVal),columns=['Mv_' + sub for sub in labelsList])

#training on sparse metadata
lr.fit(traindata[traindata.columns[17:-13]], traindata['Label']) 
Mstr=pd.DataFrame(lr.decision_function(traindata[traindata.columns[17:-13]]),columns=['Ms_' + sub for sub in labelsList])
Ms=pd.DataFrame(lr.decision_function(valdata[valdata.columns[17:-13]]),columns=['Ms_' + sub for sub in labelsList])

lr.fit(traindata[traindata.columns[-13:]], traindata['Label']) 
Sentitr=pd.DataFrame(lr.decision_function(traindata[traindata.columns[-13:]]),columns=['Senti_' + sub for sub in labelsList])
Senti=pd.DataFrame(lr.decision_function(valdata[valdata.columns[-13:]]),columns=['Senti_' + sub for sub in labelsList])


In [ ]:
df1=pd.concat([Str,Mvtr,Mstr,Sentitr],axis=1)
df2=pd.concat([S,Mv,Ms,Senti],axis=1)

In [ ]:
lr_meta=LogisticRegression()
lr_meta.fit(df1,traindata['Label'])
pred=lr_meta.predict(df2)

print(classification_report(valdata['Label'], pred,target_names=labelsList,digits=4))
print(np.mean(pred== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2603    0.2405    0.2500       237
       false     0.2648    0.2890    0.2764       263
   half-true     0.2500    0.2540    0.2520       248
 mostly-true     0.2521    0.2351    0.2433       251
  pants-fire     0.3294    0.2414    0.2786       116
        true     0.2271    0.2781    0.2500       169

    accuracy                         0.2570      1284
   macro avg     0.2639    0.2563    0.2584      1284
weighted avg     0.2595    0.2570    0.2571      1284

0.2570093457943925


### LR

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(train, traindata['Label']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['Label'], lr.predict(val),target_names=labelsList,digits=4))
print(np.mean(lr.predict(val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2066    0.1479    0.1724       169
       false     0.2512    0.2766    0.2633       188
   half-true     0.2406    0.2446    0.2426       184
 mostly-true     0.2919    0.3262    0.3081       187
  pants-fire     0.1556    0.0986    0.1207        71
        true     0.1875    0.2308    0.2069       130

    accuracy                         0.2368       929
   macro avg     0.2222    0.2208    0.2190       929
weighted avg     0.2330    0.2368    0.2329       929

0.23681377825618946


In [ ]:
print(classification_report(testdata['Label'], lr.predict(test),target_names=labelsList,digits=4))
np.mean(lr.predict(test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1947    0.1457    0.1667       151
       false     0.2476    0.2842    0.2646       183
   half-true     0.2249    0.2398    0.2321       196
 mostly-true     0.2670    0.2989    0.2821       184
  pants-fire     0.1379    0.0784    0.1000        51
        true     0.2484    0.2453    0.2468       159

    accuracy                         0.2370       924
   macro avg     0.2201    0.2154    0.2154       924
weighted avg     0.2321    0.2370    0.2330       924



0.237012987012987

## S+MJ

In [ ]:
meta_features =['Context', 'Subject', 'Job Title']
vec_St=CountVectorizer()
vec_C=CountVectorizer()
vec_S=CountVectorizer()
vec_Jt=CountVectorizer()
vec_J=CountVectorizer()

Train_St = vec_St.fit_transform(traindata['Statement'])
Val_St = vec_St.transform(valdata['Statement'])
Test_St = vec_St.transform(testdata['Statement'])

Train_C = vec_C.fit_transform(traindata['Context'])
Val_C = vec_C.transform(valdata['Context'])
Test_C = vec_C.transform(testdata['Context'])

Train_S = vec_S.fit_transform(traindata['Subject'])
Val_S = vec_S.transform(valdata['Subject'])
Test_S = vec_S.transform(testdata['Subject'])

Train_Jt = vec_Jt.fit_transform(traindata['Job Title'])
Val_Jt = vec_Jt.transform(valdata['Job Title'])
Test_Jt = vec_Jt.transform(testdata['Job Title'])

Train_J = vec_J.fit_transform(traindata['Justification'])
Val_J = vec_J.transform(valdata['Justification'])
Test_J = vec_J.transform(testdata['Justification'])

train=hstack([Train_St,Train_C,Train_S,Train_Jt,Train_J])
val=hstack([Val_St,Val_C,Val_S,Val_Jt,Val_J])
test=hstack([Test_St,Test_C,Test_S,Test_Jt,Test_J])

train= hstack([train,traindata[traindata.columns[17:]]])
val= hstack([val,valdata[valdata.columns[17:]]])
test= hstack([test,testdata[testdata.columns[17:]]])

mTrain=hstack([Train_C,Train_S,Train_Jt])
mVal=hstack([Val_C,Val_S,Val_Jt])
mTest=hstack([Test_C,Test_S,Test_Jt])

### SVM

In [ ]:
SVM = svm.LinearSVC()

In [ ]:
SVM.fit(train, traindata['Label']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['Label'], SVM.predict(val),target_names=labelsList,digits=4))
print(np.mean(SVM.predict(val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2550    0.2152    0.2334       237
       false     0.2428    0.2548    0.2486       263
   half-true     0.2621    0.2621    0.2621       248
 mostly-true     0.3128    0.3028    0.3077       251
  pants-fire     0.2679    0.2586    0.2632       116
        true     0.1854    0.2249    0.2032       169

    accuracy                         0.2547      1284
   macro avg     0.2543    0.2531    0.2530      1284
weighted avg     0.2571    0.2547    0.2553      1284

0.2546728971962617


In [ ]:
print(classification_report(testdata['Label'], SVM.predict(test),target_names=labelsList,digits=4))
np.mean(SVM.predict(test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1111    0.0861    0.0970       151
       false     0.2328    0.2404    0.2366       183
   half-true     0.2260    0.2398    0.2327       196
 mostly-true     0.2439    0.2717    0.2571       184
  pants-fire     0.1087    0.0980    0.1031        51
        true     0.2138    0.2138    0.2138       159

    accuracy                         0.2089       924
   macro avg     0.1894    0.1917    0.1900       924
weighted avg     0.2036    0.2089    0.2057       924



0.20887445887445888

### LR

In [ ]:
lr = LogisticRegression()


In [ ]:
lr.fit(train, traindata['Label']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['Label'], lr.predict(val),target_names=labelsList,digits=4))
print(np.mean(lr.predict(val)== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2397    0.1716    0.2000       169
       false     0.2770    0.3138    0.2943       188
   half-true     0.2487    0.2663    0.2572       184
 mostly-true     0.3020    0.3262    0.3136       187
  pants-fire     0.2653    0.1831    0.2167        71
        true     0.2245    0.2538    0.2383       130

    accuracy                         0.2626       929
   macro avg     0.2595    0.2525    0.2533       929
weighted avg     0.2614    0.2626    0.2599       929

0.26264800861141013


In [ ]:
print(classification_report(testdata['Label'], lr.predict(test),target_names=labelsList,digits=4))
np.mean(lr.predict(test)== testdata['Label'])

              precision    recall  f1-score   support

 barely-true     0.1818    0.1457    0.1618       151
       false     0.2700    0.2951    0.2820       183
   half-true     0.2243    0.2449    0.2341       196
 mostly-true     0.2386    0.2554    0.2467       184
  pants-fire     0.0909    0.0588    0.0714        51
        true     0.2264    0.2264    0.2264       159

    accuracy                         0.2273       924
   macro avg     0.2053    0.2044    0.2037       924
weighted avg     0.2223    0.2273    0.2240       924



0.22727272727272727

### Testing ensemble


In [ ]:
lr.fit(sTrain, traindata['Label']) 
Str=pd.DataFrame(lr.decision_function(sTrain),columns=['S_' + sub for sub in labelsList])
S=pd.DataFrame(lr.decision_function(sVal),columns=['S_' + sub for sub in labelsList])

#training on metadata countVectors
lr.fit(mTrain, traindata['Label']) 
Mvtr=pd.DataFrame(lr.decision_function(mTrain),columns=['Mv_' + sub for sub in labelsList])
Mv=pd.DataFrame(lr.decision_function(mVal),columns=['Mv_' + sub for sub in labelsList])

#training on sparse metadata
lr.fit(traindata[traindata.columns[17:-13]], traindata['Label']) 
Mstr=pd.DataFrame(lr.decision_function(traindata[traindata.columns[17:-13]]),columns=['Ms_' + sub for sub in labelsList])
Ms=pd.DataFrame(lr.decision_function(valdata[valdata.columns[17:-13]]),columns=['Ms_' + sub for sub in labelsList])

#training on sentimental analysis of data

lr.fit(traindata[traindata.columns[-13:]], traindata['Label']) 
Sentitr=pd.DataFrame(lr.decision_function(traindata[traindata.columns[-13:]]),columns=['Senti_' + sub for sub in labelsList])
Senti=pd.DataFrame(lr.decision_function(valdata[valdata.columns[-13:]]),columns=['Senti_' + sub for sub in labelsList])

#training on justification
lr.fit(jTrain, traindata['Label']) 
Jtr=pd.DataFrame(lr.decision_function(jTrain),columns=['J_' + sub for sub in labelsList])
J=pd.DataFrame(lr.decision_function(jVal),columns=['J_' + sub for sub in labelsList])


In [ ]:
df1=pd.concat([Str,Mvtr,Mstr,Sentitr,Jtr],axis=1)
df2=pd.concat([S,Mv,Ms,Senti,J],axis=1)

In [ ]:
lr_meta=LogisticRegression()
lr_meta.fit(df1,traindata['Label'])
pred=lr_meta.predict(df2)

print(classification_report(valdata['Label'], pred,target_names=labelsList,digits=4))
print(np.mean(pred== valdata['Label']))

              precision    recall  f1-score   support

 barely-true     0.2579    0.2068    0.2295       237
       false     0.2606    0.3042    0.2807       263
   half-true     0.2635    0.2944    0.2781       248
 mostly-true     0.3184    0.3108    0.3145       251
  pants-fire     0.3000    0.2069    0.2449       116
        true     0.2108    0.2308    0.2203       169

    accuracy                         0.2671      1284
   macro avg     0.2685    0.2590    0.2613      1284
weighted avg     0.2690    0.2671    0.2662      1284

0.26713395638629284


# Binary Classification

## S condition

In [ ]:
#unigram features
cvec = CountVectorizer()
X_train = cvec.fit_transform(traindata['Statement'])
X_val = cvec.transform(valdata['Statement'])
X_test = cvec.transform(testdata['Statement'])

In [ ]:
#tf-idf weighted unigram features
tfidf = TfidfVectorizer()
Xt_train = tfidf.fit_transform(traindata['Statement'])
Xt_val = tfidf.transform(valdata['Statement'])
Xt_test = tfidf.transform(testdata['Statement'])

In [ ]:
# Glove word embeddings
word2vec_output_file = 'glove.6B.50d' +'.word2vec'
glove2word2vec('/content/gdrive/My Drive/DM project/glove.6B.50d.txt', word2vec_output_file)
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
vectorizer = Word2VecVectorizer(model)

Loading in word vectors...
Finished loading in word vectors


In [ ]:
Xg_train = vectorizer.fit_transform(traindata['Statement'])
Xg_val = vectorizer.transform(valdata['Statement'])
Xg_test = vectorizer.transform(testdata['Statement'])

Numer of samples with no words found: 0 / 7212
Numer of samples with no words found: 0 / 929
Numer of samples with no words found: 0 / 924


### SVM

In [ ]:
SVM = svm.LinearSVC()

In [ ]:
SVM.fit(X_train, traindata['is_fake']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['is_fake'], SVM.predict(X_val),target_names=is_fakeList,digits=4))
print(np.mean(SVM.predict(X_val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5330    0.5093    0.5209       428
        true     0.5962    0.6188    0.6072       501

    accuracy                         0.5684       929
   macro avg     0.5646    0.5641    0.5641       929
weighted avg     0.5671    0.5684    0.5675       929

0.5683530678148547


In [ ]:
print(classification_report(testdata['is_fake'], SVM.predict(X_test),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(X_test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.4960    0.4805    0.4881       385
        true     0.6370    0.6512    0.6440       539

    accuracy                         0.5801       924
   macro avg     0.5665    0.5659    0.5661       924
weighted avg     0.5783    0.5801    0.5791       924



0.5800865800865801

In [ ]:
SVM.fit(Xt_train, traindata['is_fake'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['is_fake'], SVM.predict(Xt_val),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(Xt_val)== valdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5500    0.4626    0.5025       428
        true     0.5958    0.6766    0.6336       501

    accuracy                         0.5780       929
   macro avg     0.5729    0.5696    0.5681       929
weighted avg     0.5747    0.5780    0.5732       929



0.5780409041980624

In [ ]:
print(classification_report(testdata['is_fake'], SVM.predict(Xt_test),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(Xt_test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5513    0.4883    0.5179       385
        true     0.6621    0.7161    0.6881       539

    accuracy                         0.6212       924
   macro avg     0.6067    0.6022    0.6030       924
weighted avg     0.6159    0.6212    0.6172       924



0.6212121212121212

In [ ]:
SVM.fit(Xg_train, traindata['is_fake'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(testdata['is_fake'], SVM.predict(Xg_test),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(Xg_test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5472    0.3013    0.3886       385
        true     0.6222    0.8219    0.7082       539

    accuracy                         0.6050       924
   macro avg     0.5847    0.5616    0.5484       924
weighted avg     0.5909    0.6050    0.5751       924



0.604978354978355

In [ ]:
print(classification_report(valdata['is_fake'], SVM.predict(Xg_val),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(Xg_val)== valdata['is_fake'])

              precision    recall  f1-score   support

       false     0.6245    0.3458    0.4451       428
        true     0.5954    0.8224    0.6907       501

    accuracy                         0.6028       929
   macro avg     0.6099    0.5841    0.5679       929
weighted avg     0.6088    0.6028    0.5776       929



0.6027987082884823

### LR

In [ ]:
lr = LogisticRegression()


In [ ]:
lr.fit(X_train, traindata['is_fake']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['is_fake'], lr.predict(X_val),target_names=is_fakeList,digits=4))
print(np.mean(lr.predict(X_val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5435    0.4673    0.5025       428
        true     0.5936    0.6647    0.6271       501

    accuracy                         0.5737       929
   macro avg     0.5685    0.5660    0.5648       929
weighted avg     0.5705    0.5737    0.5697       929

0.573735199138859


In [ ]:
print(classification_report(testdata['is_fake'], lr.predict(X_test),target_names=is_fakeList,digits=4))
np.mean(lr.predict(X_test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5470    0.4987    0.5217       385
        true     0.6632    0.7050    0.6835       539

    accuracy                         0.6190       924
   macro avg     0.6051    0.6019    0.6026       924
weighted avg     0.6148    0.6190    0.6161       924



0.6190476190476191

In [ ]:
lr.fit(Xt_train, traindata['is_fake'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['is_fake'], lr.predict(Xt_val),target_names=is_fakeList,digits=4))
np.mean(lr.predict(Xt_val)== valdata['is_fake'])

              precision    recall  f1-score   support

       false     0.6157    0.3855    0.4741       428
        true     0.6021    0.7944    0.6850       501

    accuracy                         0.6060       929
   macro avg     0.6089    0.5900    0.5796       929
weighted avg     0.6084    0.6060    0.5879       929



0.6060279870828849

In [ ]:
print(classification_report(testdata['is_fake'], lr.predict(Xt_test),target_names=is_fakeList,digits=4))
np.mean(lr.predict(Xt_test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5752    0.3974    0.4700       385
        true     0.6474    0.7904    0.7118       539

    accuracy                         0.6266       924
   macro avg     0.6113    0.5939    0.5909       924
weighted avg     0.6173    0.6266    0.6111       924



0.6266233766233766

In [ ]:
lr.fit(Xg_train, traindata['is_fake'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(testdata['is_fake'], lr.predict(Xg_test),target_names=is_fakeList,digits=4))
np.mean(lr.predict(Xg_test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5409    0.3091    0.3934       385
        true     0.6222    0.8126    0.7047       539

    accuracy                         0.6028       924
   macro avg     0.5815    0.5609    0.5491       924
weighted avg     0.5883    0.6028    0.5750       924



0.6028138528138528

In [ ]:
print(classification_report(valdata['is_fake'], lr.predict(Xg_val),target_names=is_fakeList,digits=4))
np.mean(lr.predict(Xg_val)== valdata['is_fake'])

              precision    recall  f1-score   support

       false     0.6192    0.3458    0.4438       428
        true     0.5942    0.8184    0.6885       501

    accuracy                         0.6006       929
   macro avg     0.6067    0.5821    0.5661       929
weighted avg     0.6057    0.6006    0.5758       929



0.6006458557588805

## SJ


In [ ]:
svec = CountVectorizer()
sTrain = svec.fit_transform(traindata['Statement'])
sVal = svec.transform(valdata['Statement'])
sTest = svec.transform(testdata['Statement'])

jvec = CountVectorizer()
jTrain = jvec.fit_transform(traindata['Justification'])
jVal = jvec.transform(valdata['Justification'])
jTest = jvec.transform(testdata['Justification'])

train=hstack([sTrain,jTrain])
val=hstack([sVal,jVal])
test=hstack([sTest,jTest])

### SVM

In [ ]:
SVM=svm.LinearSVC()

In [ ]:
SVM.fit(train, traindata['is_fake']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['is_fake'], SVM.predict(val),target_names=is_fakeList,digits=4))
print(np.mean(SVM.predict(val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5210    0.4930    0.5066       428
        true     0.5859    0.6128    0.5990       501

    accuracy                         0.5576       929
   macro avg     0.5534    0.5529    0.5528       929
weighted avg     0.5560    0.5576    0.5564       929

0.5575888051668461


In [ ]:
print(classification_report(testdata['is_fake'], SVM.predict(test),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.4907    0.4805    0.4856       385
        true     0.6344    0.6438    0.6390       539

    accuracy                         0.5758       924
   macro avg     0.5625    0.5622    0.5623       924
weighted avg     0.5745    0.5758    0.5751       924



0.5757575757575758

### LR


In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(sTrain, traindata['is_fake']) 
Str=pd.DataFrame(lr.decision_function(sTrain))
#lr.fit(sVal, valdata['Label']) 
S=pd.DataFrame(lr.decision_function(sVal))


lr.fit(jTrain, traindata['is_fake']) 
Jtr=pd.DataFrame(lr.decision_function(jTrain))
#lr.fit(jVal, valdata['Label']) 
J=pd.DataFrame(lr.decision_function(jVal))

In [ ]:
df1=pd.concat([Str,Jtr],axis=1)
df2=pd.concat([S,J],axis=1)

In [ ]:
lr_meta=LogisticRegression()
lr_meta.fit(df1,traindata['is_fake'])
pred=lr_meta.predict(df2)

print(classification_report(valdata['is_fake'], pred,target_names=is_fakeList,digits=4))
print(np.mean(pred== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5493    0.5065    0.5270       616
        true     0.5754    0.6168    0.5954       668

    accuracy                         0.5639      1284
   macro avg     0.5624    0.5616    0.5612      1284
weighted avg     0.5629    0.5639    0.5626      1284

0.5638629283489096


In [ ]:
lr.fit(train, traindata['is_fake']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['is_fake'], lr.predict(val),target_names=is_fakeList,digits=4))
print(np.mean(lr.predict(val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5302    0.4720    0.4994       428
        true     0.5876    0.6427    0.6139       501

    accuracy                         0.5640       929
   macro avg     0.5589    0.5573    0.5566       929
weighted avg     0.5611    0.5640    0.5612       929

0.5640473627556513


In [ ]:
print(classification_report(testdata['is_fake'], lr.predict(test),target_names=is_fakeList,digits=4))
np.mean(lr.predict(test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.4986    0.4649    0.4812       385
        true     0.6354    0.6660    0.6504       539

    accuracy                         0.5823       924
   macro avg     0.5670    0.5655    0.5658       924
weighted avg     0.5784    0.5823    0.5799       924



0.5822510822510822

## S+M

In [ ]:
meta_features =['Context', 'Subject', 'Job Title']
vec_St=CountVectorizer()
vec_C=CountVectorizer()
vec_S=CountVectorizer()
vec_Jt=CountVectorizer()

Train_St = vec_St.fit_transform(traindata['Statement'])
Val_St = vec_St.transform(valdata['Statement'])
Test_St = vec_St.transform(testdata['Statement'])

Train_C = vec_C.fit_transform(traindata['Context'])
Val_C = vec_C.transform(valdata['Context'])
Test_C = vec_C.transform(testdata['Context'])

Train_S = vec_S.fit_transform(traindata['Subject'])
Val_S = vec_S.transform(valdata['Subject'])
Test_S = vec_S.transform(testdata['Subject'])

Train_Jt = vec_Jt.fit_transform(traindata['Job Title'])
Val_Jt = vec_Jt.transform(valdata['Job Title'])
Test_Jt = vec_Jt.transform(testdata['Job Title'])

train=hstack([Train_St,Train_C,Train_S,Train_Jt])
val=hstack([Val_St,Val_C,Val_S,Val_Jt])
test=hstack([Test_St,Test_C,Test_S,Test_Jt])

train= hstack([train,traindata[traindata.columns[17:]]])
val= hstack([val,valdata[valdata.columns[17:]]])
test= hstack([test,testdata[testdata.columns[17:]]])

### SVM

In [ ]:
SVM = svm.LinearSVC()

In [ ]:
SVM.fit(train, traindata['is_fake']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['is_fake'], SVM.predict(val),target_names=is_fakeList,digits=4))
print(np.mean(SVM.predict(val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5575    0.5327    0.5448       428
        true     0.6154    0.6387    0.6268       501

    accuracy                         0.5899       929
   macro avg     0.5864    0.5857    0.5858       929
weighted avg     0.5887    0.5899    0.5890       929

0.589881593110872


In [ ]:
print(classification_report(testdata['is_fake'], SVM.predict(test),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5282    0.5351    0.5316       385
        true     0.6648    0.6586    0.6617       539

    accuracy                         0.6071       924
   macro avg     0.5965    0.5968    0.5967       924
weighted avg     0.6079    0.6071    0.6075       924



0.6071428571428571

### LR

In [ ]:
lr = LogisticRegression()


In [ ]:
lr.fit(train, traindata['is_fake']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['is_fake'], lr.predict(val),target_names=is_fakeList,digits=4))
print(np.mean(lr.predict(val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.6160    0.5397    0.5753       428
        true     0.6444    0.7126    0.6768       501

    accuracy                         0.6329       929
   macro avg     0.6302    0.6261    0.6261       929
weighted avg     0.6313    0.6329    0.6300       929

0.6329386437029063


In [ ]:
print(classification_report(testdata['is_fake'], lr.predict(test),target_names=is_fakeList,digits=4))
np.mean(lr.predict(test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5482    0.5169    0.5321       385
        true     0.6684    0.6957    0.6818       539

    accuracy                         0.6212       924
   macro avg     0.6083    0.6063    0.6070       924
weighted avg     0.6183    0.6212    0.6194       924



0.6212121212121212

## S+MJ

In [ ]:
meta_features =['Context', 'Subject', 'Job Title']
vec_St=CountVectorizer()
vec_C=CountVectorizer()
vec_S=CountVectorizer()
vec_Jt=CountVectorizer()
vec_J=CountVectorizer()

Train_St = vec_St.fit_transform(traindata['Statement'])
Val_St = vec_St.transform(valdata['Statement'])
Test_St = vec_St.transform(testdata['Statement'])

Train_C = vec_C.fit_transform(traindata['Context'])
Val_C = vec_C.transform(valdata['Context'])
Test_C = vec_C.transform(testdata['Context'])

Train_S = vec_S.fit_transform(traindata['Subject'])
Val_S = vec_S.transform(valdata['Subject'])
Test_S = vec_S.transform(testdata['Subject'])

Train_Jt = vec_Jt.fit_transform(traindata['Job Title'])
Val_Jt = vec_Jt.transform(valdata['Job Title'])
Test_Jt = vec_Jt.transform(testdata['Job Title'])

Train_J = vec_J.fit_transform(traindata['Justification'])
Val_J = vec_J.transform(valdata['Justification'])
Test_J = vec_J.transform(testdata['Justification'])

train=hstack([Train_St,Train_C,Train_S,Train_Jt,Train_J])
val=hstack([Val_St,Val_C,Val_S,Val_Jt,Val_J])
test=hstack([Test_St,Test_C,Test_S,Test_Jt,Test_J])

train= hstack([train,traindata[traindata.columns[17:]]])
val= hstack([val,valdata[valdata.columns[17:]]])
test= hstack([test,testdata[testdata.columns[17:]]])

mTrain=hstack([Train_C,Train_S,Train_Jt])
mVal=hstack([Val_C,Val_S,Val_Jt])
mTest=hstack([Test_C,Test_S,Test_Jt])

### SVM

In [ ]:
SVM = svm.LinearSVC()

In [ ]:
SVM.fit(train, traindata['is_fake']) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
print(classification_report(valdata['is_fake'], SVM.predict(val),target_names=is_fakeList,digits=4))
print(np.mean(SVM.predict(val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5556    0.5023    0.5276       428
        true     0.6070    0.6567    0.6309       501

    accuracy                         0.5856       929
   macro avg     0.5813    0.5795    0.5792       929
weighted avg     0.5833    0.5856    0.5833       929

0.5855758880516685


In [ ]:
print(classification_report(testdata['is_fake'], SVM.predict(test),target_names=is_fakeList,digits=4))
np.mean(SVM.predict(test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5137    0.4883    0.5007       385
        true     0.6470    0.6698    0.6582       539

    accuracy                         0.5942       924
   macro avg     0.5803    0.5790    0.5794       924
weighted avg     0.5914    0.5942    0.5925       924



0.5941558441558441

### LR

In [ ]:
lr = LogisticRegression()

In [ ]:
#testing ensemble stacking technique

lr.fit(sTrain, traindata['is_fake']) 
Str=pd.DataFrame(lr.decision_function(sTrain))
S=pd.DataFrame(lr.decision_function(sVal))

#training on metadata countVectors
lr.fit(mTrain, traindata['is_fake']) 
Mvtr=pd.DataFrame(lr.decision_function(mTrain))
Mv=pd.DataFrame(lr.decision_function(mVal))

#training on sparse metadata
lr.fit(traindata[traindata.columns[17:-13]], traindata['is_fake']) 
Mstr=pd.DataFrame(lr.decision_function(traindata[traindata.columns[17:-13]]))
Ms=pd.DataFrame(lr.decision_function(valdata[valdata.columns[17:-13]]))

#training on sentimental analysis of data

lr.fit(traindata[traindata.columns[-13:]], traindata['is_fake']) 
Sentitr=pd.DataFrame(lr.decision_function(traindata[traindata.columns[-13:]]))
Senti=pd.DataFrame(lr.decision_function(valdata[valdata.columns[-13:]]))

#training on justification
lr.fit(jTrain, traindata['is_fake']) 
Jtr=pd.DataFrame(lr.decision_function(jTrain))
J=pd.DataFrame(lr.decision_function(jVal))


In [ ]:
df1=pd.concat([Str,Mvtr,Mstr,Sentitr,Jtr],axis=1)
df2=pd.concat([S,Mv,Ms,Senti,J],axis=1)

In [ ]:
lr_meta=LogisticRegression()
lr_meta.fit(df1,traindata['is_fake'])
pred=lr_meta.predict(df2)

print(classification_report(valdata['is_fake'], pred,target_names=is_fakeList,digits=4))
print(np.mean(pred== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5626    0.5179    0.5393       616
        true     0.5858    0.6287    0.6065       668

    accuracy                         0.5755      1284
   macro avg     0.5742    0.5733    0.5729      1284
weighted avg     0.5747    0.5755    0.5743      1284

0.5755451713395638


In [ ]:
lr.fit(train, traindata['is_fake']) 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(classification_report(valdata['is_fake'], lr.predict(val),target_names=is_fakeList,digits=4))
print(np.mean(lr.predict(val)== valdata['is_fake']))

              precision    recall  f1-score   support

       false     0.5891    0.5327    0.5595       428
        true     0.6310    0.6826    0.6558       501

    accuracy                         0.6136       929
   macro avg     0.6101    0.6077    0.6077       929
weighted avg     0.6117    0.6136    0.6114       929

0.6135629709364908


In [ ]:
print(classification_report(testdata['is_fake'], lr.predict(test),target_names=is_fakeList,digits=4))
np.mean(lr.predict(test)== testdata['is_fake'])

              precision    recall  f1-score   support

       false     0.5380    0.4961    0.5162       385
        true     0.6591    0.6957    0.6769       539

    accuracy                         0.6126       924
   macro avg     0.5985    0.5959    0.5966       924
weighted avg     0.6086    0.6126    0.6099       924



0.6125541125541125

# Other Methods tested

In [ ]:
from scipy.sparse import coo_matrix, hstack
# create the transform
vec = CountVectorizer()
vecj= CountVectorizer()

# tokenize, build vocab and encode training data
traindata_vec = vec.fit_transform(traindata['Statement'].map(str))
traindata_vecj = vecj.fit_transform(traindata['Justification'].map(str))
print(traindata_vec.shape)
print(traindata_vecj.shape)

union1 = FeatureUnion([('S1', vec), ('J1', vecj)])
train=hstack([traindata_vec,traindata_vecj])
print(train.shape)
#union.shape
testdata_vec = vec.transform(testdata['Statement'].map(str))
testdata_vecj = vecj.transform(testdata['Justification'].map(str))
union2 = FeatureUnion([('S2', vec), ('J2', vecj)])
test=hstack([testdata_vec,testdata_vecj])
#print(testdata_vec.shape)


(10240, 11601)
(10240, 22329)
(10240, 33930)


In [ ]:
###six-way classification###

### concatenating statement and justification ###
traindata['New'] = traindata['Statement'].map(str) + traindata['Justification'].map(str)
testdata['New'] = testdata['Statement'].map(str) + testdata['Justification'].map(str)
X_train=traindata[['Statement','Justification']]
Y_train=traindata[['is_fake']]
test=testdata[['Statement','Justification']]
print(X_train.shape)
print(Y_train.shape)



(10240, 2)
(10240, 1)


In [ ]:

nvec = CountVectorizer(ngram_range=(1,1))
traindata_nvec = nvec.fit_transform(traindata['New'].values)

logReg_pipeline_cv = Pipeline([
    ('LogRCV', nvec),
    ('LogR_model', LogisticRegression())
])

logReg_pipeline_cv.fit(traindata['New'], traindata['Label'])
predictions_logReg = logReg_pipeline_cv.predict(testdata['New'])
logReg_cv = np.mean(predictions_logReg == testdata['Label'])
print(logReg_cv)



0.1925808997632202


In [ ]:
logReg_pipeline_cv.fit(traindata['New'], traindata['Label'])
predictions_logReg = logReg_pipeline_cv.predict(testdata['Statement'])
logReg_cv = np.mean(predictions_logReg == testdata['Label'])
print(logReg_cv)

0.22888713496448304
